In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import akita_utils_forplotting as utils
import numpy as np
import pandas as pd
import sys
import os, psutil
import time 
import matplotlib.pyplot as plt

from optparse import OptionParser


nt = ['A', 'T', 'C', 'G']

In [ ]:
#file_path='/pollard/data/projects/shzhang_rotation/somatic_variants/somatic_structural_variants/'

#vcf_path= file_path+ '02dc6e6c-1baf-4f43-8360-d4950838a63a.somaticSV.vcf.gz'

# #input_dir=options.in_dir
input_dir='/pollard/home/shzhang/akita/results/20230326/bedpe/'


#vcf_name='02dc6e6c-1baf-4f43-8360-d4950838a63a.somaticSV.vcf.gz'
vcf_name='35c87b1a-eea4-4841-bed3-4c9591f5f417.somaticSV.bedpe'
vcf_path=input_dir + vcf_name
print(vcf_path)


In [ ]:
# Read variants 
variants = pd.read_csv(vcf_path, sep='\t')
#print('num SVs:' + str(len(variants)))
print('Sample {} has {} SVs'.format(vcf_name, str(len(variants))))

variants['SVTYPE']=variants['INFO_A'].str.extract('SVTYPE=(.*?)\;')


#subset to non BND since BND are taken care of 
#variants=variants[variants.TYPE != 'BND']
variants=variants[variants.TYPE =='DUP']
variants=variants[variants.SPAN >5e4]
variants=variants[variants.SPAN <5e5]

#variants=variants[variants.SVTYPE != 'BND']
variants = variants.reset_index(drop=True)
#pd.to_numeric(variants['MSE'])
#variants=variants[variants.MSE >0.5]
#variants[variants.]
#variants[variants.START_B==14981062]
variants


In [ ]:
i=2


#2 is lopsided
#3 is not 

 # Get info on variant
variant = variants.loc[i]

CHR = 'chr'+variant.CHROM_A
POS = variant.START_A
REF = variant.REF_A
ALT = variant.ALT_A
SVTYPE = variant.TYPE
END= (int(variant.END_B)) if SVTYPE!='BND' else np.nan
print(variant.SPAN)

#print(variant)

    
    
if all([x in nt for x in ALT]):
    REF_mat, ALT_mat  = utils.get_scores(CHR, POS, REF, ALT, SVTYPE, shift='none')
    #MSE_left, CORR_left, MSE_track_left, CORR_track_left = utils.get_scores(CHR, POS, REF, ALT, SVTYPE, shift='left')
    #MSE_right, CORR_right, MSE_track_right, CORR_track_right = utils.get_scores(CHR, POS, REF, ALT, SVTYPE, shift='right')


else:
    #tracks are  of shape (448,)
    #for each sample need to return (448 x 6 x n) array 
    REF_mat, ALT_mat, REF_mat_before, ALT_mat_before = utils.get_scores_SV(CHR, POS, ALT, END, SVTYPE, shift='none')
    #MSE_left, CORR_left, MSE_track_left, CORR_track_left = utils.get_scores_SV(CHR, POS, ALT, END, SVTYPE, shift='left')
    #MSE_right, CORR_right, MSE_track_right, CORR_track_right = utils.get_scores_SV(CHR, POS, ALT, END, SVTYPE, shift='right')
        

        


In [ ]:
        
plt.figure(figsize=(8,4))
target_index = 0
vmin=-2; vmax=2



plt.subplot(121) 
im = plt.matshow(ALT_mat, fignum=False, cmap= 'RdBu_r', vmax=vmax, vmin=vmin)
plt.colorbar(im, fraction=.04, pad = 0.05, ticks=[-2,-1, 0, 1,2]);
plt.title('ALT Prediction' + '\n' + '{} at {}:{}-{}'.format(SVTYPE, CHR, POS, END),y=1.15 )
#plt.title('pred test',y=1.15 )

#plt.ylabel(myseq_str)

# plot target 
plt.subplot(122) 
im = plt.matshow(REF_mat, fignum=False, cmap= 'RdBu_r', vmax=vmax, vmin=vmin)
plt.colorbar(im, fraction=.04, pad = 0.05, ticks=[-2,-1, 0, 1,2]);
#plt.title( 'target-'+str(hic_num_to_name_dict[target_index]),y=1.15)
plt.title('REF Target' + '\n' + '{} at {}:{}-{}'.format(SVTYPE, CHR, POS, END),y=1.15 )



plt.tight_layout()
#plt.show()
#plt.savefig('/pollard/home/shzhang/akita/figs/del_mask_schematic.pdf', dpi=300)   # save the figure to file